#### Libraries

In [1]:
import dspy
import numpy as np
from dotenv import load_dotenv
from dspy.datasets import DataLoader
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShotWithRandomSearch, LabeledFewShot

from src.starling import StarlingLM # <- Custom Local Model Client for Starling7B

_ = load_dotenv()

#### LLM

The model that will be used in this notebook is [Starling7B](https://huggingface.co/Nexusflow/Starling-LM-7B-beta), and the evaluation model will be [GPT-4 Turbo](https://openai.com/gpt-4).

In [2]:
# Share generation args between models
generation_args = {
    "temperature":0,
    "max_tokens":500,
    "stop":"\n\n",
    "model_type":"chat",
    "n": 1
}
# Model specific args
model_info = {
    "gpt-4": {"model": "gpt-4-0125-preview", "api_base": "https://api.openai.com/v1/"},
    "starling": {"model": "Nexusflow/Starling-LM-7B-beta"}
}

In [3]:
# Set up the models
lm = StarlingLM(**model_info["starling"], **generation_args)
evaluator_lm = dspy.OpenAI(**model_info["gpt-4"], **generation_args)

dspy.configure(lm=lm)

In [4]:
# Testing inference of Starling
lm("What is the capital of Colombia?")

[' The capital of Colombia is Bogotá. It is the largest city in the country and serves as the political, economic, and cultural center of Colombia. Located in the Andean region of the country, Bogotá has a rich history and is known for its vibrant arts scene, diverse architecture, and numerous museums and cultural institutions.']

#### Load dataset

The dataset that will be used in this notebook is [gretelai/synthetic_text_to_sql](https://huggingface.co/datasets/gretelai/synthetic_text_to_sql)

In [5]:
# Define random seed
np.random.seed(1399)

In [6]:
# Load dataset
dl = DataLoader()
dataset = dl.from_huggingface(
    dataset_name="gretelai/synthetic_text_to_sql", # Dataset name from Huggingface
    fields=("sql_prompt", "sql_context", "sql"), # Fields needed
    input_keys=("sql_prompt", "sql_context") # What our model expects to recieve to generate an output
)
trainset, testset = dl.sample(dataset["train"], n=50), dl.sample(dataset["test"], n=25) # 50 training samples, 25 testing samples

In [7]:
# Verify an example of the dataset
sample = dl.sample(dataset=trainset, n=1)[0]
for k, v in sample.items():
    print(f"\n{k.upper()}:\n")
    print(v)


SQL_PROMPT:

What are the top 5 most purchased lipsticks based on sales volume across all regions?

SQL_CONTEXT:

CREATE TABLE lipsticks (lipstick_id INT, lipstick_name VARCHAR(100), region VARCHAR(50), sales_volume INT); INSERT INTO lipsticks (lipstick_id, lipstick_name, region, sales_volume) VALUES (1, 'Ruby Woo', 'North America', 1500), (2, 'Russian Red', 'Europe', 1200), (3, 'Cherry', 'Asia', 1800), (4, 'Lady Danger', 'South America', 1000), (5, 'Mademoiselle', 'Australia', 1600);

SQL:

SELECT lipstick_name, SUM(sales_volume) FROM lipsticks GROUP BY lipstick_name ORDER BY SUM(sales_volume) DESC LIMIT 5;


#### Signature (Input/Output)

In [8]:
class TextToSql(dspy.Signature):
    """Transform a natural language query into a SQL query."""

    sql_prompt = dspy.InputField(desc="Natural language query")
    sql_context = dspy.InputField(desc="Context for the query")
    sql = dspy.OutputField(desc="SQL query")

### Inference

#### Baseline Inference

In [9]:
generate_sql_query = dspy.Predict(signature=TextToSql)

result = generate_sql_query(
    sql_prompt=sample["sql_prompt"],
    sql_context=sample["sql_context"]
)

for k, v in result.items():
    print(f"\n{k.upper()}:\n")
    print(v)


SQL:

SELECT lipstick_name, SUM(sales_volume) as total_sales
FROM lipsticks
GROUP BY lipstick_name
ORDER BY total_sales DESC
LIMIT 5;


#### ChainOfThought Inference

In [10]:
generate_sql_query = dspy.ChainOfThought(signature=TextToSql)

result = generate_sql_query(
    sql_prompt=sample["sql_prompt"],
    sql_context=sample["sql_context"]
)

for k, v in result.items():
    print(f"\n{k.upper()}:\n")
    print(v)


RATIONALE:

produce the SQL query. We need to find the top 5 most purchased lipsticks based on sales volume across all regions.

SQL:

SELECT lipstick_name, SUM(sales_volume) as total_sales
FROM lipsticks
GROUP BY lipstick_name
ORDER BY total_sales DESC
LIMIT 5;


### Metric of evaluation

#### Metric definition

In [12]:
class Correctness(dspy.Signature):
    """Assess if the SQL query accurately answers the given natural language query based on the provided context."""

    sql_prompt = dspy.InputField(desc="Natural language query ")
    sql_context = dspy.InputField(desc="Context for the query")
    sql = dspy.InputField(desc="SQL query")
    correct = dspy.OutputField(desc="Indicate whether the SQL query correctly answers the natural language query based on the given context", prefix="Yes/No:")

In [13]:
def correctness_metric(example, pred, trace=None):
    sql_prompt, sql_context, sql = example.sql_prompt, example.sql_context, pred.sql

    correctness = dspy.Predict(Correctness)

    with dspy.context(lm=evaluator_lm): 
        correct = correctness(
            sql_prompt=sql_prompt,
            sql_context=sql_context,
            sql=sql,
        )
    
    score = int(correct.correct=="Yes")

    if trace is not None:
        return score == 1

    return score

#### Evaluate single data point

In [19]:
_correctness = correctness_metric(
    example=sample,
    pred=result
)
print(f"Correct SQL query: {'Yes' if _correctness else 'No'}")

Correct SQL query: Yes


In [20]:
evaluator_lm.inspect_history(n=1)





Assess if the SQL query accurately answers the given natural language query based on the provided context.

---

Follow the following format.

Sql Prompt: Natural language query

Sql Context: Context for the query

Sql: SQL query

Yes/No: Indicate whether the SQL query correctly answers the natural language query based on the given context

---

Sql Prompt: What are the top 5 most purchased lipsticks based on sales volume across all regions?

Sql Context: CREATE TABLE lipsticks (lipstick_id INT, lipstick_name VARCHAR(100), region VARCHAR(50), sales_volume INT); INSERT INTO lipsticks (lipstick_id, lipstick_name, region, sales_volume) VALUES (1, 'Ruby Woo', 'North America', 1500), (2, 'Russian Red', 'Europe', 1200), (3, 'Cherry', 'Asia', 1800), (4, 'Lady Danger', 'South America', 1000), (5, 'Mademoiselle', 'Australia', 1600);

Sql: SELECT lipstick_name, SUM(sales_volume) as total_sales FROM lipsticks GROUP BY lipstick_name ORDER BY total_sales DESC LIMIT 5;

Yes/No: Yes





#### Evaluate entire dataset - GPT 3.5

<div style="background-color: #F0F0F0; padding: 10px; border-radius: 5px;"> <p style="color: #4B4B4B; font-size: 18px; font-weight: bold; margin: 0;"> 📊 Baseline Result </p> <p style="color: #4B4B4B; font-size: 16px; margin: 5px 0 0;"> Without any optimization, <strong>GPT 3.5 Turbo</strong> achieves a <strong>68% correctness</strong> in the evaluation of 25 test samples. </p> </div>

In [33]:
with dspy.context(lm=dspy.OpenAI(model="gpt-3.5-turbo-0125", api_base="https://api.openai.com/v1/", **generation_args)):
    evaluate = Evaluate(devset=testset, metric=correctness_metric, num_threads=10, display_progress=True, display_table=5)
    evaluate(generate_sql_query)

Average Metric: 17 / 25  (68.0): 100%|██████████| 25/25 [00:07<00:00,  3.53it/s]

Average Metric: 17 / 25  (68.0%)



/home/jjmov99/dspy-testing/.venv/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)
/home/jjmov99/dspy-testing/.venv/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:263: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '1' '0' '1' '1']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,sql_prompt,sql_context,example_sql,rationale,pred_sql,correctness_metric
0,What is the maximum depth of the Arctic Ocean?,"CREATE TABLE oceans (name TEXT, max_depth REAL); INSERT INTO oceans (name, max_depth) VALUES ('Pacific Ocean', 10994), ('Atlantic Ocean', 8605), ('Indian Ocean', 7490), ('Southern Ocean', 7280),...",SELECT max_depth FROM oceans WHERE name = 'Arctic Ocean';,find the maximum depth of the Arctic Ocean. We need to select the maximum depth from the oceans table where the name is 'Arctic Ocean'.,SELECT max_depth FROM oceans WHERE name = 'Arctic Ocean';,1
1,How many access to justice initiatives were implemented per country?,"CREATE TABLE countries (country_id INT, country VARCHAR(50)); INSERT INTO countries (country_id, country) VALUES (1, 'USA'), (2, 'Canada'), (3, 'Mexico'), (4, 'Brazil'), (5, 'Argentina'); CREATE TABLE...","SELECT c.country, COUNT(i.id) AS total_initiatives FROM initiatives i JOIN countries c ON i.country_id = c.country_id WHERE i.implemented = TRUE GROUP BY c.country;","produce the SQL. We need to count the number of initiatives that were implemented per country. To do this, we will join the countries table...","SELECT c.country, COUNT(i.id) AS num_initiatives FROM countries c JOIN initiatives i ON c.country_id = i.country_id WHERE i.implemented = TRUE GROUP BY c.country;",1
2,What is the total loan amount disbursed by socially responsible lending organizations in the United States?,"CREATE TABLE tlad_orgs (org_name TEXT, loan_amount NUMERIC); INSERT INTO tlad_orgs (org_name, loan_amount) VALUES ('Socially Responsible USA', 2000000), ('Lending with Integrity', 2500000), ('Fair Lending Inc', 1800000);","SELECT SUM(loan_amount) FROM tlad_orgs WHERE org_name IN ('Socially Responsible USA', 'Lending with Integrity', 'Fair Lending Inc') AND country = 'USA';",produce the sql. We need to filter the lending organizations that are socially responsible and then calculate the total loan amount disbursed by them.,SELECT SUM(loan_amount) AS total_loan_amount FROM tlad_orgs WHERE org_name LIKE '%Socially Responsible%';,0
3,What was the total R&D expenditure for each organization in H1 2022?,"CREATE TABLE rd_expenditure (expenditure_id INT, organization_id INT, quarter INT, year INT, amount DECIMAL(10, 2));","SELECT organization_id, SUM(amount) as total_expenditure FROM rd_expenditure WHERE quarter IN (1, 2) AND year = 2022 GROUP BY organization_id;",produce the sql. We need to filter the data for the first half of 2022 (H1 2022) and then calculate the total R&D expenditure for...,"SELECT organization_id, SUM(amount) AS total_rd_expenditure FROM rd_expenditure WHERE year = 2022 AND quarter <= 2 GROUP BY organization_id;",1
4,Which countries participated in Mars missions in the 2010s?,"CREATE TABLE Mars_Missions (Mission_ID INT, Mission_Name VARCHAR(50), Country VARCHAR(50), Year INT, PRIMARY KEY (Mission_ID)); INSERT INTO Mars_Missions (Mission_ID, Mission_Name, Country, Year) VALUES (1, 'Phoenix', 'United...",SELECT DISTINCT Country FROM Mars_Missions WHERE Year BETWEEN 2010 AND 2019;,produce the sql. We need to select distinct countries from the Mars_Missions table where the year is in the 2010s.,SELECT DISTINCT Country FROM Mars_Missions WHERE Year >= 2010 AND Year <= 2019;,1


#### Evaluate entire dataset - Starling7B

<div style="background-color: #FFCCCB; padding: 10px; border-radius: 5px;"> <p style="color: #8B0000; font-size: 18px; font-weight: bold; margin: 0;"> ⚠️ Evaluation Result </p> <p style="color: #8B0000; font-size: 16px; margin: 5px 0 0;"> Without any optimization, <strong>Starling7B</strong> achieves a <strong>60% correctness</strong> in the evaluation of 25 test samples. </p> </div>

In [21]:
evaluate = Evaluate(devset=testset, metric=correctness_metric, num_threads=10, display_progress=True, display_table=5)
evaluate(generate_sql_query)

Average Metric: 15 / 25  (60.0): 100%|██████████| 25/25 [00:13<00:00,  1.82it/s]

Average Metric: 15 / 25  (60.0%)



/home/jjmov99/dspy-testing/.venv/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)
/home/jjmov99/dspy-testing/.venv/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:263: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '1' '0' '1' '1']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,sql_prompt,sql_context,example_sql,rationale,pred_sql,correctness_metric
0,What is the maximum depth of the Arctic Ocean?,"CREATE TABLE oceans (name TEXT, max_depth REAL); INSERT INTO oceans (name, max_depth) VALUES ('Pacific Ocean', 10994), ('Atlantic Ocean', 8605), ('Indian Ocean', 7490), ('Southern Ocean', 7280),...",SELECT max_depth FROM oceans WHERE name = 'Arctic Ocean';,"produce the SQL query. We first need to identify the table that contains the information about the oceans and their maximum depths. In this case,...",SELECT max_depth FROM oceans WHERE name = 'Arctic Ocean';,1
1,How many access to justice initiatives were implemented per country?,"CREATE TABLE countries (country_id INT, country VARCHAR(50)); INSERT INTO countries (country_id, country) VALUES (1, 'USA'), (2, 'Canada'), (3, 'Mexico'), (4, 'Brazil'), (5, 'Argentina'); CREATE TABLE...","SELECT c.country, COUNT(i.id) AS total_initiatives FROM initiatives i JOIN countries c ON i.country_id = c.country_id WHERE i.implemented = TRUE GROUP BY c.country;","produce the SQL query. We first need to identify the relevant tables and columns in the context provided. In this case, we have two tables:...","SELECT c.country, COUNT(i.id) AS initiatives_implemented FROM countries c JOIN initiatives i ON c.country_id = i.country_id WHERE i.implemented = TRUE GROUP BY c.country;",1
2,What is the total loan amount disbursed by socially responsible lending organizations in the United States?,"CREATE TABLE tlad_orgs (org_name TEXT, loan_amount NUMERIC); INSERT INTO tlad_orgs (org_name, loan_amount) VALUES ('Socially Responsible USA', 2000000), ('Lending with Integrity', 2500000), ('Fair Lending Inc', 1800000);","SELECT SUM(loan_amount) FROM tlad_orgs WHERE org_name IN ('Socially Responsible USA', 'Lending with Integrity', 'Fair Lending Inc') AND country = 'USA';",produce the SQL query. We need to:,"1. Identify the relevant table and columns: In this case, the relevant table is `tlad_orgs` with columns `org_name` and `loan_amount`. 2. Filter the data for...",0
3,What was the total R&D expenditure for each organization in H1 2022?,"CREATE TABLE rd_expenditure (expenditure_id INT, organization_id INT, quarter INT, year INT, amount DECIMAL(10, 2));","SELECT organization_id, SUM(amount) as total_expenditure FROM rd_expenditure WHERE quarter IN (1, 2) AND year = 2022 GROUP BY organization_id;","produce the SQL query. We need to find the total R&D expenditure for each organization in H1 2022, which corresponds to the first two quarters...","SELECT organization_id, SUM(amount) as total_expenditure FROM rd_expenditure WHERE quarter IN (1, 2) AND year = 2022 GROUP BY organization_id;",1
4,Which countries participated in Mars missions in the 2010s?,"CREATE TABLE Mars_Missions (Mission_ID INT, Mission_Name VARCHAR(50), Country VARCHAR(50), Year INT, PRIMARY KEY (Mission_ID)); INSERT INTO Mars_Missions (Mission_ID, Mission_Name, Country, Year) VALUES (1, 'Phoenix', 'United...",SELECT DISTINCT Country FROM Mars_Missions WHERE Year BETWEEN 2010 AND 2019;,produce the SQL query. We need to find the countries that participated in Mars missions in the 2010s.,SELECT DISTINCT Country FROM Mars_Missions WHERE Year BETWEEN 2010 AND 2019;,1


60.0

### Optimize for Text2SQL

#### Create program

In [22]:
# Define the program ~ You can think of this a Pytorch model.
class TextToSqlProgram(dspy.Module):
    def __init__(self):
        super().__init__()
        self.program = dspy.ChainOfThought(signature=TextToSql)
    
    def forward(self, sql_prompt, sql_context):
        return self.program(
            sql_prompt=sql_prompt,
            sql_context=sql_context
        )

### FewShot

In [23]:
# Execute the optimizer -> this only adds few shots to the prompt
optimizer = LabeledFewShot(k=4)
optmized_program = optimizer.compile(student=TextToSqlProgram(), trainset=trainset)

In [24]:
optmized_program(sql_context=sample["sql_context"], sql_prompt=sample["sql_prompt"])

Prediction(
    rationale='produce the SQL query. We want to find the top 5 most purchased lipsticks based on sales volume across all regions.',
    sql='SELECT lipstick_name, SUM(sales_volume) as total_sales\nFROM lipsticks\nGROUP BY lipstick_name\nORDER BY total_sales DESC\nLIMIT 5;'
)

#### What is happening inside?

In [25]:
lm.inspect_history(n=1)





Transform a natural language query into a SQL query.

---

Sql Prompt: Identify the top 2 water consuming industries in California in 2021.
Sql Context: CREATE TABLE industrial_water_usage (state VARCHAR(20), year INT, sector VARCHAR(30), usage FLOAT); INSERT INTO industrial_water_usage (state, year, sector, usage) VALUES ('California', 2021, 'Agriculture', 45612.3), ('California', 2021, 'Manufacturing', 34567.2), ('California', 2021, 'Mining', 23456.1), ('California', 2021, 'Gasoline Production', 12345.0), ('California', 2021, 'Food Processing', 56789.0);
Sql: SELECT sector, usage FROM industrial_water_usage WHERE state = 'California' AND year = 2021 ORDER BY usage DESC LIMIT 2;

Sql Prompt: What are the textile sourcing countries with a sustainability rating greater than 75?
Sql Context: CREATE TABLE sourcing (id INT, country VARCHAR(20), sustainability_rating INT); INSERT INTO sourcing (id, country, sustainability_rating) VALUES (1, 'China', 70); INSERT INTO sourcing (id, countr

#### Evaluate the optimized program


<div style="background-color: #FFF8DC; padding: 10px; border-radius: 5px;"> <p style="color: #DAA520; font-size: 18px; font-weight: bold; margin: 0;"> 🌟 Evaluation Progress </p> <p style="color: #DAA520; font-size: 16px; margin: 5px 0 0;"> With Few Shot Optimization, <strong>Starling7B</strong> achieves a <strong>72.0% correctness</strong> in the evaluation of 25 test samples. </p> </div>

In [26]:
evaluate(optmized_program)

Average Metric: 18 / 25  (72.0): 100%|██████████| 25/25 [00:14<00:00,  1.72it/s]

Average Metric: 18 / 25  (72.0%)



/home/jjmov99/dspy-testing/.venv/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)
/home/jjmov99/dspy-testing/.venv/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:263: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '1' '1' '0' '1']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,sql_prompt,sql_context,example_sql,rationale,pred_sql,correctness_metric
0,What is the maximum depth of the Arctic Ocean?,"CREATE TABLE oceans (name TEXT, max_depth REAL); INSERT INTO oceans (name, max_depth) VALUES ('Pacific Ocean', 10994), ('Atlantic Ocean', 8605), ('Indian Ocean', 7490), ('Southern Ocean', 7280),...",SELECT max_depth FROM oceans WHERE name = 'Arctic Ocean';,"produce the SQL query. First, we need to identify the table and column that contains the information about the maximum depth of the oceans. In...",SELECT max_depth FROM oceans WHERE name = 'Arctic Ocean';,1
1,How many access to justice initiatives were implemented per country?,"CREATE TABLE countries (country_id INT, country VARCHAR(50)); INSERT INTO countries (country_id, country) VALUES (1, 'USA'), (2, 'Canada'), (3, 'Mexico'), (4, 'Brazil'), (5, 'Argentina'); CREATE TABLE...","SELECT c.country, COUNT(i.id) AS total_initiatives FROM initiatives i JOIN countries c ON i.country_id = c.country_id WHERE i.implemented = TRUE GROUP BY c.country;",produce the SQL query. We need to count the number of implemented initiatives per country.,"SELECT c.country, COUNT(i.id) AS implemented_initiatives FROM countries c JOIN initiatives i ON c.country_id = i.country_id WHERE i.implemented = TRUE GROUP BY c.country;",1
2,What is the total loan amount disbursed by socially responsible lending organizations in the United States?,"CREATE TABLE tlad_orgs (org_name TEXT, loan_amount NUMERIC); INSERT INTO tlad_orgs (org_name, loan_amount) VALUES ('Socially Responsible USA', 2000000), ('Lending with Integrity', 2500000), ('Fair Lending Inc', 1800000);","SELECT SUM(loan_amount) FROM tlad_orgs WHERE org_name IN ('Socially Responsible USA', 'Lending with Integrity', 'Fair Lending Inc') AND country = 'USA';","produce the SQL query. First, we need to identify the organizations that are socially responsible lending organizations in the United States. In this case, we...","SELECT SUM(loan_amount) FROM tlad_orgs WHERE org_name IN ('Socially Responsible USA', 'Lending with Integrity', 'Fair Lending Inc');",1
3,What was the total R&D expenditure for each organization in H1 2022?,"CREATE TABLE rd_expenditure (expenditure_id INT, organization_id INT, quarter INT, year INT, amount DECIMAL(10, 2));","SELECT organization_id, SUM(amount) as total_expenditure FROM rd_expenditure WHERE quarter IN (1, 2) AND year = 2022 GROUP BY organization_id;",produce the SQL query. We need to find the total R&D expenditure for each organization in the first quarter (H1) of 2022.,"SELECT organization_id, SUM(amount) as total_expenditure FROM rd_expenditure WHERE quarter = 1 AND year = 2022 GROUP BY organization_id;",0
4,Which countries participated in Mars missions in the 2010s?,"CREATE TABLE Mars_Missions (Mission_ID INT, Mission_Name VARCHAR(50), Country VARCHAR(50), Year INT, PRIMARY KEY (Mission_ID)); INSERT INTO Mars_Missions (Mission_ID, Mission_Name, Country, Year) VALUES (1, 'Phoenix', 'United...",SELECT DISTINCT Country FROM Mars_Missions WHERE Year BETWEEN 2010 AND 2019;,"produce the SQL query. First, we need to identify the missions that took place in the 2010s. Then, we need to find the countries that...",SELECT DISTINCT Country FROM Mars_Missions WHERE Year BETWEEN 2010 AND 2019;,1


72.0

### BootstrapFewShotWithRandomSearch

[DSPy docs](https://dspy-docs.vercel.app/docs/building-blocks/optimizers) recommend that in a setup like the one with have at hand, with ~50 samples, the best option is to use `BootstrapFewShotWithRandomSearch`:

![image](assets/dspy.png)

In [27]:
optimizer2 = BootstrapFewShotWithRandomSearch(metric=correctness_metric, max_bootstrapped_demos=2, num_candidate_programs=8, num_threads=5)
optmized_program_2 = optimizer2.compile(student = TextToSqlProgram(), trainset=trainset, valset=testset)

Going to sample between 1 and 2 traces per predictor.
Will attempt to train 8 candidate sets.


Average Metric: 15 / 25  (60.0): 100%|██████████| 25/25 [00:20<00:00,  1.20it/s]
/home/jjmov99/dspy-testing/.venv/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


Average Metric: 15 / 25  (60.0%)
Score: 60.0 for set: [0]
New best score: 60.0 for seed -3
Scores so far: [60.0]
Best score: 60.0


Average Metric: 18 / 25  (72.0): 100%|██████████| 25/25 [00:30<00:00,  1.22s/it]


Average Metric: 18 / 25  (72.0%)
Score: 72.0 for set: [16]
New best score: 72.0 for seed -2
Scores so far: [60.0, 72.0]
Best score: 72.0


...

Average Metric: 17 / 25  (68.0%)
Score: 68.0 for set: [16]
Scores so far: [60.0, 72.0, 56.0, 72.0, 64.0, 68.0, 80.0, 76.0, 68.0, 68.0]
Best score: 80.0
Average of max per entry across top 1 scores: 0.8
Average of max per entry across top 2 scores: 0.84
Average of max per entry across top 3 scores: 0.88
Average of max per entry across top 5 scores: 0.96
Average of max per entry across top 8 scores: 1.0
Average of max per entry across top 9999 scores: 1.0


  4%|▍         | 2/50 [00:09<03:47,  4.75s/it]


Bootstrapped 2 full traces after 3 examples in round 0.


Average Metric: 18 / 25  (72.0): 100%|██████████| 25/25 [00:30<00:00,  1.23s/it]

Average Metric: 18 / 25  (72.0%)
Score: 72.0 for set: [16]
Scores so far: [60.0, 72.0, 56.0, 72.0, 64.0, 68.0, 80.0, 76.0, 68.0, 68.0, 72.0]
Best score: 80.0
Average of max per entry across top 1 scores: 0.8
Average of max per entry across top 2 scores: 0.84
Average of max per entry across top 3 scores: 0.88
Average of max per entry across top 5 scores: 0.96
Average of max per entry across top 8 scores: 1.0
Average of max per entry across top 9999 scores: 1.0
11 candidate programs found.


In [28]:
optmized_program_2(sql_context=sample["sql_context"], sql_prompt=sample["sql_prompt"])

Prediction(
    rationale='produce the SQL query. We need to find the top 5 most purchased lipsticks based on sales volume across all regions.',
    sql='SELECT lipstick_name, SUM(sales_volume) as total_sales_volume\nFROM lipsticks\nGROUP BY lipstick_name\nORDER BY total_sales_volume DESC\nLIMIT 5;'
)

In [29]:
lm.inspect_history(n=1)





Transform a natural language query into a SQL query.

---

Sql Prompt: What are the total clinical trial expenses for drug 'DrugG' in 2022?
Sql Context: CREATE TABLE clinical_trials (drug VARCHAR(50), year INT, expenses INT); INSERT INTO clinical_trials (drug, year, expenses) VALUES ('DrugG', 2022, 8000000), ('DrugH', 2022, 9000000);
Sql: SELECT expenses FROM clinical_trials WHERE drug = 'DrugG' AND year = 2022;

Sql Prompt: Insert a new record for a donation of $250 made by a donor from India to the Health category on March 15, 2022.
Sql Context: CREATE TABLE Donations (DonationID INT, DonorID INT, Amount DECIMAL, DonationDate DATE, Country TEXT); INSERT INTO Donations (DonationID, DonorID, Amount, DonationDate, Country) VALUES (1, 1, 50, '2022-01-01', 'USA'), (2, 1, 75, '2022-01-01', 'USA'), (3, 2, 100, '2022-02-01', 'Canada'), (4, 3, 200, '2022-03-01', 'Mexico'), (5, 4, 150, '2022-03-01', 'Brazil'); CREATE TABLE ProgramCategories (CategoryID INT, Category TEXT); INSERT INTO Prog

#### Evaluate the optimized program

<div style="background-color: #E0F8E0; padding: 10px; border-radius: 5px;"> <p style="color: #006400; font-size: 18px; font-weight: bold; margin: 0;"> ✅ Evaluation Success </p> <p style="color: #006400; font-size: 16px; margin: 5px 0 0;"> With BootstrapFewShotWithRandomSearch Optimization, <strong>Starling7B</strong> achieves an <strong>80.0% correctness</strong> in the evaluation of 25 test samples. </p> </div>

In [30]:
evaluate(optmized_program_2)

  0%|          | 0/25 [00:00<?, ?it/s]

Average Metric: 20 / 25  (80.0): 100%|██████████| 25/25 [00:23<00:00,  1.08it/s]

Average Metric: 20 / 25  (80.0%)



/home/jjmov99/dspy-testing/.venv/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)
/home/jjmov99/dspy-testing/.venv/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:263: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '1' '1' '1' '1']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,sql_prompt,sql_context,example_sql,rationale,pred_sql,correctness_metric
0,What is the maximum depth of the Arctic Ocean?,"CREATE TABLE oceans (name TEXT, max_depth REAL); INSERT INTO oceans (name, max_depth) VALUES ('Pacific Ocean', 10994), ('Atlantic Ocean', 8605), ('Indian Ocean', 7490), ('Southern Ocean', 7280),...",SELECT max_depth FROM oceans WHERE name = 'Arctic Ocean';,produce the SQL query. We need to find the maximum depth of the Arctic Ocean.,SELECT max_depth FROM oceans WHERE name = 'Arctic Ocean';,1
1,How many access to justice initiatives were implemented per country?,"CREATE TABLE countries (country_id INT, country VARCHAR(50)); INSERT INTO countries (country_id, country) VALUES (1, 'USA'), (2, 'Canada'), (3, 'Mexico'), (4, 'Brazil'), (5, 'Argentina'); CREATE TABLE...","SELECT c.country, COUNT(i.id) AS total_initiatives FROM initiatives i JOIN countries c ON i.country_id = c.country_id WHERE i.implemented = TRUE GROUP BY c.country;",produce the SQL query. We need to find the number of access to justice initiatives implemented per country.,"SELECT c.country, COUNT(i.id) as implemented_initiatives FROM countries c JOIN initiatives i ON c.country_id = i.country_id WHERE i.implemented = TRUE GROUP BY c.country;",1
2,What is the total loan amount disbursed by socially responsible lending organizations in the United States?,"CREATE TABLE tlad_orgs (org_name TEXT, loan_amount NUMERIC); INSERT INTO tlad_orgs (org_name, loan_amount) VALUES ('Socially Responsible USA', 2000000), ('Lending with Integrity', 2500000), ('Fair Lending Inc', 1800000);","SELECT SUM(loan_amount) FROM tlad_orgs WHERE org_name IN ('Socially Responsible USA', 'Lending with Integrity', 'Fair Lending Inc') AND country = 'USA';",produce the SQL query. We need to find the total loan amount disbursed by socially responsible lending organizations in the United States.,"SELECT SUM(loan_amount) as total_loan_amount FROM tlad_orgs WHERE org_name IN ('Socially Responsible USA', 'Lending with Integrity', 'Fair Lending Inc');",1
3,What was the total R&D expenditure for each organization in H1 2022?,"CREATE TABLE rd_expenditure (expenditure_id INT, organization_id INT, quarter INT, year INT, amount DECIMAL(10, 2));","SELECT organization_id, SUM(amount) as total_expenditure FROM rd_expenditure WHERE quarter IN (1, 2) AND year = 2022 GROUP BY organization_id;",produce the SQL query. We need to find the total R&D expenditure for each organization in the first half of 2022 (H1 2022).,"SELECT organization_id, SUM(amount) as total_expenditure FROM rd_expenditure WHERE quarter BETWEEN 1 AND 2 AND year = 2022 GROUP BY organization_id;",1
4,Which countries participated in Mars missions in the 2010s?,"CREATE TABLE Mars_Missions (Mission_ID INT, Mission_Name VARCHAR(50), Country VARCHAR(50), Year INT, PRIMARY KEY (Mission_ID)); INSERT INTO Mars_Missions (Mission_ID, Mission_Name, Country, Year) VALUES (1, 'Phoenix', 'United...",SELECT DISTINCT Country FROM Mars_Missions WHERE Year BETWEEN 2010 AND 2019;,produce the SQL query. We need to find the countries that participated in Mars missions in the 2010s.,SELECT DISTINCT Country FROM Mars_Missions WHERE Year BETWEEN 2010 AND 2019;,1


80.0